In [1]:
import os, sys
from os import path
sys.path.append(os.path.abspath(path.join(os.getcwd(), os.pardir, 'src')))

import project_config as pc

from data import utils as data_utils

import re
import pickle
import numpy as np
import pandas as pd

In [2]:
for i in range(0, 500, 250):
    print(i)

0
250


In [2]:
df = data_utils.load_nyc_listings()

In [3]:
listing_embedder = ListingEmbedder(device='cuda:0')

In [4]:
embeddings = listing_embedder.from_dataframe(df, 400)

100%|█████████████████████████████████████████| 109/109 [06:58<00:00,  3.84s/it]


In [5]:
embeddings.shape

(43566, 1024)

In [12]:
embeddings

array([[-0.01053734,  0.02992929, -0.02854822, ..., -0.02058447,
        -0.02488858, -0.00536096],
       [-0.01013899,  0.0089483 , -0.02855307, ..., -0.02224569,
        -0.01729184, -0.01087915],
       [ 0.00449256,  0.01541421, -0.0204808 , ..., -0.01774506,
        -0.02749335, -0.01489215],
       ...,
       [ 0.01111768,  0.01973216, -0.02297438, ..., -0.01283255,
        -0.03752194, -0.00829629],
       [-0.00870882,  0.01095518, -0.02508694, ..., -0.02465589,
        -0.02248235, -0.0080701 ],
       [-0.00099459,  0.01030623, -0.0142141 , ..., -0.02049175,
        -0.01991566, -0.01334427]], dtype=float32)

In [6]:
scores = embeddings @ embeddings.T

In [7]:
np.mean(scores)

0.95169115

In [25]:
df.iloc[1]

id                                                                           5121
listing_url                                     https://www.airbnb.com/rooms/5121
room_type                                                            Private room
neighbourhood_group_cleansed                                             Brooklyn
neighbourhood_cleansed                                         Bedford-Stuyvesant
bathrooms_text                                                                NaN
bedrooms                                                                        0
beds                                                                            1
accommodates                                                                    2
price                                                                        60.0
latitude                                                                 40.68535
longitude                                                               -73.95512
property_type   

In [26]:
df.iloc[6389]

id                                                                       10167436
listing_url                                 https://www.airbnb.com/rooms/10167436
room_type                                                            Private room
neighbourhood_group_cleansed                                             Brooklyn
neighbourhood_cleansed                                               Williamsburg
bathrooms_text                                                      1 shared bath
bedrooms                                                                        0
beds                                                                            1
accommodates                                                                    2
price                                                                        50.0
latitude                                                                 40.70904
longitude                                                               -73.94147
property_type   

In [14]:
def filter_by_price(df):
    prices = df.price.to_numpy()
    prices[prices == 0] = 1
    log_prices = np.log10(prices)
    return np.abs(log_prices[:, None] - log_prices) <= 0.3

In [58]:
# filter_by_price(df)

array([[ True, False, False, ..., False,  True, False],
       [False,  True, False, ..., False, False,  True],
       [False, False,  True, ...,  True, False,  True],
       ...,
       [False, False,  True, ...,  True, False,  True],
       [ True, False, False, ..., False,  True, False],
       [False,  True,  True, ...,  True, False,  True]])

In [15]:
def filter_by_neighborhood(df):
    neighbourhoods = df.neighbourhood_cleansed.to_numpy()
    return neighbourhoods[:, None] != neighbourhoods

In [60]:
# filter_by_neighborhood(df)

array([[False,  True,  True, ...,  True,  True,  True],
       [ True, False,  True, ...,  True,  True,  True],
       [ True,  True, False, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ..., False,  True,  True],
       [ True,  True,  True, ...,  True, False,  True],
       [ True,  True,  True, ...,  True,  True, False]])

In [16]:
def apply_heuristic_filters(scores, df):
    scores *= filter_by_price(df)
    scores *= filter_by_neighborhood(df)
    scores[scores < 0.9] = 0
    return scores

In [17]:
scores = apply_heuristic_filters(scores, df)

In [28]:
scores

array([[0.       , 0.       , 0.       , ..., 0.9432435, 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.9575317,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.9432435, 0.       , 0.       , ..., 0.       , 0.       ,
        0.9580718],
       [0.       , 0.9575317, 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.9580718, 0.       ,
        0.       ]], dtype=float32)

In [37]:
ranking = np.argsort(scores, axis=1)[:, -10:]

In [38]:
ranking = ranking[:, ::-1]

In [ ]:
df.apply()

In [43]:
df.iloc[ranking[0]].id.tolist()

[18393354,
 49059730,
 9357,
 21935551,
 881333052483713671,
 9474607,
 2695372,
 50335735,
 5110541,
 850380530005995874]

In [81]:
df['embedding'] = df.apply(lambda row: pickle.dumps(embeddings[row.name].tolist()), axis=1)

In [79]:
df['similar_listings'] = df.apply(lambda row: pickle.dumps(df.iloc[ranking[row.name]].id.tolist()), axis=1)

In [80]:
df[df.id.eq(16338086)].iloc[0]

id                                                                       16338086
listing_url                                 https://www.airbnb.com/rooms/16338086
room_type                                                         Entire home/apt
neighbourhood_group_cleansed                                             Brooklyn
neighbourhood_cleansed                                  Prospect-Lefferts Gardens
bathrooms_text                                                             1 bath
bedrooms                                                                        2
beds                                                                            2
accommodates                                                                    4
price                                                                       180.0
latitude                                                                 40.66122
longitude                                                               -73.94466
property_type   

In [62]:
df[df.id.eq(774131904854449024)].iloc[0]

id                                                             774131904854449024
listing_url                       https://www.airbnb.com/rooms/774131904854449024
room_type                                                         Entire home/apt
neighbourhood_group_cleansed                                             Brooklyn
neighbourhood_cleansed                                              Crown Heights
bathrooms_text                                                             1 bath
bedrooms                                                                        2
beds                                                                            2
accommodates                                                                    4
price                                                                       140.0
latitude                                                                 40.67241
longitude                                                               -73.96142
property_type   

In [84]:
listings = list(df.iloc[0:50].T.to_dict().values())

In [90]:
", :".join(list(listings[0].keys()))

'id, :listing_url, :room_type, :neighbourhood_group_cleansed, :neighbourhood_cleansed, :bathrooms_text, :bedrooms, :beds, :accommodates, :price, :latitude, :longitude, :property_type, :description, :neighborhood_overview, :host_about, :embedding, :similar_listings'